# Algorytmy macierzowe - zadanie nr 3 - Formaty macierzy rzadkich

" Proszę wybrać swój ulubiony język programowania. Proszę wziąć największe macierze, które używali Państwo w zadaniu pierwszym (ewentualnie zmniejszyć p żeby stały się rzadsze). Proszę przekonwertować je do stosownego formatu i przemnożyć
przez siebie.

6. Pierwsza i druga macierz CSR format "

Marcin Hawryluk, Norbert Wolniak <br>grupa: piątek 12:50B <hr>

In [1]:
import numpy as np
from time import time
import pandas as pd
import os

## Generowanie macierzy

In [2]:
def read_matrix(file_name):
    with open(file_name, 'r') as file:
        for line in file:
            if line.strip() == '':
                continue
            if line[0] == '#':
                if line[2:6] == "rows":
                    _, _, size = line.split()
                    size = int(size)
                    matrix = np.zeros((size, size))
            else:
                row, col, val = line.split(' ')
                matrix[int(row)-1, int(col)-1] = val
            
    return matrix

## Format CSR

In [3]:
def convert_to_csr(matrix):
    m, n = matrix.shape
    ICL = []
    VAL = []
    ROWPTR = []
    counter = 0
    for i in range(n): # rows
        ROWPTR.append(counter)
        for j in range(m): # columns
            val_ij = matrix[i,j]
            if abs(val_ij) < 1e-8:
                continue
            ICL.append(j)
            VAL.append(val_ij)
            counter += 1
    ROWPTR.append(counter)
    
    return ICL, VAL, ROWPTR

In [105]:
test_matrix_A = np.array([[0,0,1],
                          [0,2,0],
                          [3,1,4]])
test_matrix_B = np.array([[0,0,1],
                          [0,2,0],
                          [3,1,4]])

In [106]:
print(test_matrix_A @ test_matrix_B)

[[ 3  1  4]
 [ 0  4  0]
 [12  6 19]]


In [107]:
ICL, VAL, ROWPTR = convert_to_csr(test_matrix_A)
print("ICL : ", ICL)
print("VAL : ", VAL)
print("ROWPTR : ", ROWPTR)

ICL :  [2, 1, 0, 1, 2]
VAL :  [1, 2, 3, 1, 4]
ROWPTR :  [0, 1, 2, 5]


In [108]:
A = convert_to_csr(test_matrix_A)
B = convert_to_csr(test_matrix_B)

## Mnożenie macierzy

In [116]:
def get_ICL_VAL_from_dict(C_ROW):
    C_ROW_sorted = sorted(C_ROW.items(), key=lambda item : item[0]) # sorting by key (column index)
    ICL_C = [x[0] for x in C_ROW_sorted] # column indexes list
    VAL_C = [x[1] for x in C_ROW_sorted] # values list
    return ICL_C, VAL_C
    
def matmul_CSR(A, B):
    ICL_A, VAL_A, ROWPTR_A = A
    ICL_B, VAL_B, ROWPTR_B = B    
    ICL_C, VAL_C, ROWPTR_C = [], [], []

    elements_counter = 0
    for i in range(len(ROWPTR_A) - 1):
        row_a_start = ROWPTR_A[i]
        row_a_end = ROWPTR_A[i+1]
        C_ROW = dict() # dict to store results of row * row multiplication instead of row * col
        for j in range(row_a_start,row_a_end):
            col_a = ICL_A[j]
            val_a = VAL_A[j]
            row_b_start = ROWPTR_B[col_a]    
            row_b_end = ROWPTR_B[col_a + 1]
            for p in range(row_b_start,row_b_end):
                col_b = ICL_B[p]
                val_b = VAL_B[p]
                C_ROW[col_b] = C_ROW.get(col_b, 0) + (val_a * val_b)
                
        ICL, VAL = get_ICL_VAL_from_dict(C_ROW)
        ICL_C += ICL
        VAL_C += VAL
        ROWPTR_C.append(elements_counter)
        elements_counter += len(VAL)
    
    ROWPTR_C.append(elements_counter)
    return ICL_C, VAL_C, ROWPTR_C

In [112]:
C = matmul_CSR(A, B)

In [92]:
def print_CSR_matrix(A):
    ICL, VAL, ROWPTR = A
    VAL = VAL.copy()
    m_col = max(ICL) + 1
    for i in range(len(ROWPTR)-1):
        for p in range(m_col):
            if p in ICL[ROWPTR[i]:ROWPTR[i+1]]:
                print(VAL.pop(0), end=",")
            else:
                print(0, end=",")
        print("\n")

In [110]:
print_CSR_matrix(C)

3,1,4,

0,4,0,

12,6,19,



## Pomiar czasów

## Wnioski

* 
* 

M. Hawryluk, N. Wolniak. 2021